<a href="https://colab.research.google.com/github/Machine-Learning-Tokyo/DL-workshop-series/blob/master/ConvNets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Code for plotting the keras model graph
Uncomment -> run -> comment out -> restart runtime

In [0]:
# %%capture
# !pip install pydot
# !apt-get install graphviz
# !sed -i 's/def _check_pydot()/def _check__pydot()/g' /usr/local/lib/python3.6/dist-packages/keras/utils/vis_utils.py
# !sed -i 's/_check_pydot()/#_check_pydot()/g' /usr/local/lib/python3.6/dist-packages/keras/utils/vis_utils.py

### [Cheatsheet](https://drive.google.com/open?id=1iWhJiMT9pgWqYA_3-iRyvQ1DwlhV3hGdR-pinZiiHfk)

### **Disclaimer**:

These functions were developed for teaching reasons.

The main idea is to keep the code as simple and clean as possible for someone to get the structure of each model easily.

For more configurable implementations of these models please refer to other available github repositories.

## Imports

In [0]:
%%capture
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, DepthwiseConv2D, SeparableConv2D
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D
from keras.layers import BatchNormalization, concatenate, add, Dropout, ReLU, Lambda, Activation, LeakyReLU

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from time import time
import numpy as np

## AlexNet

In [0]:
def alexnet(input_shape, n_classes):
  input = Input(input_shape)
  
  # actually batch normalization didn't exist back then
  # they used LRN (Local Response Normalization) for regularization
  x = Conv2D(96, 11, strides=4, padding='same', activation='relu')(input)
  x = BatchNormalization()(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = Conv2D(256, 5, padding='same', activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
  x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
  x = Conv2D(256, 3, strides=1, padding='same', activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = Flatten()(x)
  x = Dense(4096, activation='relu')(x)
  x = Dense(4096, activation='relu')(x)
  
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  return model

## VGG

In [0]:
def vgg(input_shape, n_classes):
  filters = 64, 128, 256, 512, 512
  repetitions = 2, 2, 3, 3, 3
  
  def vgg_block(x, f, r):
    for _ in range(r):
      x = Conv2D(f, 3, padding='same', activation='relu')(x)
    x = MaxPool2D(2, strides=2)(x)
    return x
  
  
  input = Input(input_shape)
  
  x = input
  for f, r in zip(filters, repetitions):
    x = vgg_block(x, f, r)
  
  x = Flatten()(x)
  x = Dense(4096, activation='relu')(x)
  x = Dense(4096, activation='relu')(x)
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  return model

## Inception

In [0]:
def googlenet(input_shape, n_classes):
  
  def incpeption_block(x, f):
    t1 = Conv2D(f[0], 1, activation='relu')(x)
    
    t2 = Conv2D(f[1], 1, activation='relu')(x)
    t2 = Conv2D(f[2], 3, padding='same', activation='relu')(t2)
    
    t3 = Conv2D(f[3], 1, activation='relu')(x)
    t3 = Conv2D(f[4], 5, padding='same', activation='relu')(t3)
    
    t4 = MaxPool2D(3, 1, padding='same')(x)
    t4 = Conv2D(f[5], 1, activation='relu')(t4)
    
    output = concatenate([t1, t2, t3, t4])
    return output
  
  
  input = Input(input_shape)
  
  x = Conv2D(64, 7, strides=2, padding='same', activation='relu')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = Conv2D(64, 1, activation='relu')(x)
  x = Conv2D(192, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = inception_block(x, [64, 96, 128, 16, 32, 32])
  x = inception_block(x, [128, 128, 192, 32, 96, 64])
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = inception_block(x, [192, 96, 208, 16, 48, 64])
  x = inception_block(x, [160, 112, 224, 24, 64, 64])
  x = inception_block(x, [128, 128, 256, 24, 64, 64])
  x = inception_block(x, [112, 144, 288, 32, 64, 64])
  x = inception_block(x, [256, 160, 32, 32, 128, 128])
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = inception_block(x, [256, 160, 320, 32, 128, 128])
  x = inception_block(x, [384, 192, 384, 48, 128, 128])
  
  x = AvgPool2D(7, strides=1)(x)
  x = Dropout(0.4)(x)
  
  x = Flatten()(x)
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  return model

## MobileNet

In [0]:
def mobilenet(input_shape, n_classes):
  
  def conv_bn_rl(x, f, k=1, s=1, p='same'):
    x = Conv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
  
  
  def mobilenet_block(x, f, s=1):
    x = DepthwiseConv2D(3, s, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = conv_bn_rl(x, f)
    return x
  
  
  input = Input(input_shape)
  
  x = conv_bn_rl(input, 32, 3, 2)
  x = mobilenet_block(x, 64)
  
  x = mobilenet_block(x, 128, 2)
  x = mobilenet_block(x, 128)
  
  x = mobilenet_block(x, 256, 2)
  x = mobilenet_block(x, 256)
  
  x = mobilenet_block(x, 512, 2)
  for i in range(5):
    x = mobilenet_block(x, 512)
  
  x = mobilenet_block(x, 1024, 2)
  x = mobilenet_block(x, 1024)
  
  x = AvgPool2D(7)(x)
  
  x = Flatten()(x)
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  return model

## ShuffleNet

In [0]:
def shufflenet(input_shape, n_classes, g=8):
  channels = 384, 769, 1536
  repetitions = 3, 7, 3
  
  def ch_shuffle(x, g):
#     1 2 3 4 5 6 7 8 9 -reshape-> 1 2 3 -permute dims-> 1 4 7 -reshape-> 1 4 7 2 5 8 3 6 9
#                                  4 5 6                 2 5 8
#                                  7 8 9                 3 6 9
    
    _, w, h, ch = K.int_shape(x)
    ch_g = ch // g
    
    def shuffle_op(x):
      x = K.reshape(x, [-1, w, h, ch_g, g])
      x = K.permute_dimensions(x, [0, 1, 2, 4, 3])
      x = K.reshape(x, [-1, w, h, ch])
      return x
    
    x = Lambda(shuffle_op)(x)
    return x
    
  
  def gconv(tensor, ch, g):
    _, _, _, in_ch = K.int_shape(tensor)
    ch_g = in_ch // g
    out_ch = ch // g
    group = []
    
    for i in range(g):
#       x = tensor[:, :, :, i*ch_g:(i+1)*ch_g]
      x = Lambda(lambda x: x[:, :, :, i*ch_g: (i+1)*ch_g])(tensor)
      x = Conv2D(out_ch, 1)(x)
      group.append(x)
    
    x = concatenate(group)
    return x
  
  
  def shufflenet_block(tensor, ch, s, g):
    x = gconv(tensor, ch // 4, g)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = ch_shuffle(x, g)
    x = DepthwiseConv2D(3, strides=s, padding='same')(x)
    x = BatchNormalization()(x)
    x = gconv(x, ch if s==1 else ch-K.int_shape(tensor)[-1], g)
    x = BatchNormalization()(x)
    
    if s == 1:
      x = add([tensor, x])
    else:
      avg = AvgPool2D(3, strides=2, padding='same')(tensor)
      x = concatenate([avg, x])
    
    output = ReLU()(x)
    return output
  
  
  def stage(x, ch, r, g):
    x = shufflenet_block(x, ch, 2, g)
    
    for i in range(r):
      x = shufflenet_block(x, ch, 1, g)
      
    return x
  
  input = Input(input_shape)
  
  x = Conv2D(24, 3, strides=2, padding='same')(input)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  for ch, r in zip(channels, repetitions):
    x = stage(x, ch, r, g)
    
  x = GlobalAvgPool2D()(x)
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  return model

## ResNet

In [0]:
def resnet(input_shape, n_classes):
  
  def conv_bn_rl(x, f, k=1, s=1, p='same'):
    x = Conv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
  
  
  def conv_block(tensor, f1, f2, s):
    x = conv_bn_rl(tensor, f1)
    x = conv_bn_rl(x, f1, 3, s=s)
    x = Conv2D(f2, 1)(x)
    x = BatchNormalization()(x)
    
    shortcut = Conv2D(f2, 1, strides=s, padding='same')(tensor)
    shortcut = BatchNormalization()(shortcut)
    
    x = add([shortcut, x])
    output = ReLU()(x)
    
    return output
  
  
  def identity_block(tensor, f1, f2):
    x = conv_bn_rl(tensor, f1)
    x = conv_bn_rl(x, f1, 3)
    x = Conv2D(f2, 1)(x)
    x = BatchNormalization()(x)
    
    x = add([tensor, x])
    output = ReLU()(x)
    
    return output
  
  
  def resnet_block(x, f1, f2, r, s=2):
    x = conv_block(x, f1, f2, s)
    
    for _ in range(r-1):
      x = identity_block(x, f1, f2)
    
    return x
  
  
  input = Input(input_shape)
  
  x = conv_bn_rl(input, 64, 7, 2)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = resnet_block(x, 64, 256, 3, 1)
  x = resnet_block(x, 128, 512, 4)
  x = resnet_block(x, 256, 1024, 6)
  x = resnet_block(x, 512, 2048, 3)
  
  x = AvgPool2D(7, strides=1)(x)
  x = Flatten()(x)
  
  output = Dense(n_classes, activation='softmax')(x)
  model = Model(input, output)
  
  return model

## DenseNet

In [0]:
def densenet(input_shape, n_classes, k=32):
  repetitions = 6, 12, 48, 32
  
  def bn_rl_conv(x, f, k=1, s=1, p='same'):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(f, k, strides=s, padding=p)(x)
    return x
  
  
  def dense_block(tensor, k):
    x = bn_rl_conv(tensor, 4*k)
    x = bn_rl_conv(x, k, 3)
    output = concatenate([tensor, x])
    return output
  
  
  def transition(x):
    x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)
    x = AvgPool2D(2, strides=2, padding='same')(x)
    return x
  
  
  def dense_blocks(x, r, k):
    for _ in range(r):
      x = dense_block(x, k)
    return x
  
  
  input = Input(input_shape)
  
  x = Conv2D(64, 7, strides=2, padding='same')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  for r in repetitions:
    d = dense_blocks(x, r, k)
    x = transition(d)
  
  x = AvgPool2D(7, strides=1)(d)
  x = Flatten()(x)
  
  output = Dense(n_classes, activation='softmax')(x)
  model = Model(input, output)
  
  return model

## Xception

In [0]:
def xception(input_shape, n_classes):
  
  def conv_bn(x, f, k=1, s=1, p='same'):
    x = Conv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    return x
  
  
  def rl_sep(x, f, k=3, s=1, p='same'):
    x = ReLU()(x)
    x = SeparableConv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    return x
  
  
  def sep_rl(x, f, k=3, s=1, p='same'):
    x = SeparableConv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
  
  
  def entry_flow(x, filters):
    res = x
    for f in filters:
      res = conv_bn(res, f, s=2)
      x = rl_sep(x, f)
      x = rl_sep(x, f)
      x = MaxPool2D(3, strides=2, padding='same')(x)
      x = add([res, x])
    return x
      
      
  def midde_flow(tensor, f):
    x = rl_sep(tensor, f)
    x = rl_sep(x, f)
    x = rl_sep(x, f)
    x = add([tensor, x])
    return x
  
  
  def exit_flow(x, f):
    res = conv_bn(x, f[1], s=2)
    
    x = rl_sep(x, f[0])
    x = rl_sep(x, f[1])
    x = MaxPool2D(3, strides=2, padding='same')(x)
    x = add([res, x])
    
    x = sep_rl(x, f[2])
    x = sep_rl(x, f[3])
    return x
  
  
  input = Input(input_shape)
  
  x = conv_bn(input, 32, 3, 2)
  x = ReLU()(x)
  x = conv_bn(input, 64, 3)
  x = ReLU()(x)
  
  x = entry_flow(x, [128, 256, 728])
  for _ in range(8):
    x = midde_flow(x, 728)
  x = exit_flow(x, [728, 1024, 1536, 2048])
  
  x = GlobalAvgPool2D()(x)
  output = Dense(n_classes, activation='softmax')(x)
  model = Model(input, output)
  
  return model

## Unet

In [0]:
def unet(input_shape=(572, 572, 1), f=64, steps=4, n_classes=2):
  
  def downstream(x, f):
    x = Conv2D(f, 3, activation='relu')(x)
    d = Conv2D(f, 3, activation='relu')(x)
    x = MaxPool2D(2, strides=2, padding='same')(d)
    return d, x
  
  
  def crop_merge(x, d):
    _, xw, xh, _ = K.int_shape(x)
    _, dw, dh, _ = K.int_shape(d)
    mw, mh = (dw-xw)//2, (dh-xh)//2
    
    d = Lambda(lambda x: x[:, mw: dw-mw, mh: dh-mh, :])(d)
    x = concatenate([d, x])
    return x
    
  
  def upstream(x, f, d):
    x = UpSampling2D()(x)
    x = Conv2D(f, 2, padding='same')(x)
    x = crop_merge(x, d)
    x = Conv2D(f, 3, activation='relu')(x)
    x = Conv2D(f, 3, activation='relu')(x)
    return x
    
  
  input = Input(input_shape)
  x = input
  
  downsampled = []
  for i in range(steps+1):
    d, x = downstream(x, f*2**i)
    downsampled.append(d)
  x = downsampled.pop()
  
  for i in range(steps-1, -1, -1):
    x = upstream(x, f*2**i, downsampled[i])
  
  output = Conv2D(n_classes, 1)(x)
  model = Model(input, output)
  
  return model

## SqueezeNet

In [0]:
def squeezenet(input_shape, n_classes):
  
  def fire_module(x, f1, f2, f3):
    x = Conv2D(f1, 1, activation='relu')(x)
    ones = Conv2D(f2, 1, activation='relu')(x)
    threes = Conv2D(f3, 3, padding='same', activation='relu')(x)
    output = concatenate([ones, threes])
    return output
  
  
  input = Input(input_shape)
  
  x = Conv2D(96, 7, strides=2, padding='same', activation='relu')(input)
  x = MaxPool2D(3, strides=2)(x)
  
  x = fire_module(x, 16, 64, 64)
  x = fire_module(x, 16, 64, 64)
  x = fire_module(x, 32, 128, 128)
  x = MaxPool2D(3, strides=2)(x)
  
  x = fire_module(x, 32, 128, 128)
  x = fire_module(x, 48, 192, 192)
  x = fire_module(x, 48, 192, 192)
  x = fire_module(x, 64, 256, 256)
  x = MaxPool2D(3, strides=2)(x)
  
  x = fire_module(x, 64, 256, 256)
  x = Conv2D(n_classes, 1)(x)
  x = GlobalAvgPool2D()(x)
  
  output = Activation('softmax')(x)
  model = Model(input, output)
  
  return model

## YOLO

In [0]:
def yolo(input_shape=(448, 448, 3), n_outputs=30):
  activation = LeakyReLU(0.1)
  
  def conv_1_3(x, f1, f2, r=1):
    for _ in range(r):
      x = Conv2D(f1, 1, padding='same', activation=activation)(x)
      x = Conv2D(f2, 3, padding='same', activation=activation)(x)
    return x
    
  
  input = Input(input_shape)
  
  x = Conv2D(64, 7, strides=2, padding='same', activation=activation)(input)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Conv2D(192, 3, padding='same', activation=activation)(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = conv_1_3(x, 128, 256)
  x = conv_1_3(x, 256, 512)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = conv_1_3(x, 256, 512, 4)
  x = conv_1_3(x, 512, 1024)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = conv_1_3(x, 512, 1024, 2)
  x = Conv2D(1024, 3, padding='same', activation=activation)(x)
  x = Conv2D(1024, 3, strides=2, padding='same', activation=activation)(x)
  
  x = Conv2D(1024, 3, padding='same', activation=activation)(x)
  x = Conv2D(1024, 3, padding='same', activation=activation)(x)
  
  x = Dense(4096, activation=activation)(x)
  output = Dense(n_outputs)(x)
  
  model = Model(input, output)
  return model

## RefineNet

In [0]:
def refinenet(backbone_model, n_classes=21, tensor_names=('add_3', 'add_7', 'add_13', 'add_16')):
  '''
  backbone_model: (pretrained, non trainable) model used to extract the features (refinenet's inputs) 
  tensor_names: list of names of the backbone_model's tensors to be used as inputs (default: names from resnet)
  '''
  
  filters = 512, 256, 256, 256
  input = backbone_model.input
  
  def rcu(tensor, f, conv_shortcut=False):
    x = ReLU()(tensor)
    x = Conv2D(f, 3, padding='same')(x)
    x = ReLU()(tensor)
    x = Conv2D(f, 3, padding='same')(x)
    
    if conv_shortcut:
      tensor = Conv2D(f, 3, padding='same')(tensor)
      
    output = add([tensor, x])
    return output
  
  
  def fusion_block(x1, x2, f):
    x1 = Conv2D(f, 3, padding='same')(x1)
    
    x2 = Conv2D(f, 3, padding='same')(x2)
    x2 = UpSampling2D()(x2)
    
    output = add([x1, x2])
    return output
  
  
  
  def chained_block(tensor, steps=2):
    f = K.int_shape(tensor)[-1]
    tensor = ReLU()(tensor)
    
    x = tensor
    for _ in range(steps):
      x = MaxPool2D(5, strides=1, padding='same')(x)
      x = Conv2D(f, 3, padding='same')(x)
      tensor = add([tensor, x])
    
    return tensor
  
  
  def refine_block(x1, x2=None, f=256):
    x = rcu(x1, f, True)
    x = rcu(x, f)
    
    if x2 is not None:
      x2 = rcu(x2, f, True)
      x2 = rcu(x2, f)
      x = fusion_block(x, x2, f)
    
    x = chained_block(x)
    
    output = rcu(x, f)
    return output
  
  
  inputs = [layer.output for layer in backbone_model.layers if layer.name in tensor_names][::-1]
  x = None
  
  for t, f in zip(inputs, filters):
    x = refine_block(t, x, f)
  
  x = rcu(x, filters[-1])
  x = rcu(x, filters[-1])
  
  output = Dense(n_classes, activation='softmax')(x)
  model = Model(input, output)
  
  return model

## Print and plot the model

In [0]:
input_shape = 224, 224, 3
n_classes = 1000

K.clear_session()
model = alexnet(input_shape, n_classes)
model.summary()

In [0]:
SVG(model_to_dot(model).create(prog='dot', format='svg'))

## Calculate inference time

In [0]:
repetitions = 10
input = np.random.randn(1, *input_shape)

output = model.predict(input)
start = time()
for _ in range(repetitions):
  output = model.predict(input)
  
print((time() - start) / repetitions)